# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [114]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [115]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [116]:
# Import the necessary CSVs to Pandas DataFrames
year_grouping = pd.read_csv(Path('../Data/year_grouping.csv'))
year_grouping_house = pd.read_csv(Path('../Data/year_grouping_house.csv'))
new_sfo_df = pd.read_csv(Path('../Data/new_sfo_df.csv'))
top_10_neighborhood = pd.read_csv(Path('../Data/top_10_neighborhood.csv'))
top_10_neighborhood_by_year = pd.read_csv(Path('../Data/top_10_neighborhood_by_year.csv'))
neighborhood_geolocation = pd.read_csv(Path('../Data/neighborhood_geolocation.csv')).round(2)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [130]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    return year_grouping_house.hvplot.bar(x='year', y='housing_units', ylim=[year_grouping_house.min() - year_grouping_house.std(), year_grouping_house.max() + year_grouping_house.std()], height=600, title='Housing Units Per Year', color='purple').opts(bgcolor='lightgray')

def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    return year_grouping['gross_rent'].hvplot.line(x='year', y='gross_rent', color='blue', title='Average Monthly Rent').opts(bgcolor='lightgray')

def average_sales_price():
    """Average Sales Price Per Year."""
    return year_grouping['sale_price_sqr_foot'].hvplot.line(x='year', y='sale_price_sqr_foot', color="green", title='Average Sale Price Per Square Foot').opts(bgcolor='lightgray')

def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    return new_sfo_df.hvplot.line(x='year', y='sale_price_sqr_foot', groupby='neighborhood', ylabel='Sale Price Per Square Foot', color='darkblue').opts(bgcolor='lightgray')

def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    return top_10_neighborhood.hvplot.bar(y='gross_rent', rot=90, height=500, ylabel='Gross Rent', xlabel='Neighborhood', title='Top 10 Expensive Neighborhoods')

def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
    return top_10_neighborhood_by_year.hvplot.bar(x='year', y=['sale_price_sqr_foot', 'gross_rent'], groupby='neighborhood', rot=70, xlabel="Year", height=600, title='Comparison of Rent and Sales Prices of Most Expensive Neighborhoods')
    
def parallel_coordinates():
    """Parallel Coordinates Plot."""
    return px.parallel_coordinates(
        top_10_neighborhood,
        dimensions=['neighborhood', 'sale_price_sqr_foot', 'housing_units', 'gross_rent'],
        color_continuous_scale=px.colors.sequential.Inferno,
        color='gross_rent'
        )

def parallel_categories():
    """Parallel Categories Plot."""
    return px.parallel_categories(
        top_10_neighborhood,
        dimensions=['neighborhood', 'sale_price_sqr_foot', 'housing_units', 'gross_rent'],
        color_continuous_scale=px.colors.sequential.Inferno,
        color='sale_price_sqr_foot'
        )

def neighborhood_map():
    """Neighborhood Map."""
    return px.scatter_mapbox(
        neighborhood_geolocation,
        lat="Lat",
        lon="Lon",
        size='sale_price_sqr_foot',
        color=neighborhood_geolocation.index,
        zoom=10
        )

def sunburst():
    """Sunburst Plot."""
    return px.sunburst(
        top_10_neighborhood_by_year,
        path=['year', 'neighborhood'],
        values='gross_rent'
        )

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [131]:
# Set Rows and Columns
geospatial = pn.Column(
    "# Price Visualized Through Neighborhood and Yearly Grouping",
    pn.Row(neighborhood_map()) # sunburst() will not render due to bug explained in README
)

parallels = pn.Column(
    "# Correlations Between Averages and Neighborhoods",
    parallel_categories(),
    parallel_coordinates()
)

by_neighborhood = pn.Column(
    "# Averages by Neighborhood",
    average_price_by_neighborhood()
)

averages = pn.Column(
    "# Graphical Visualization of Overall Averages",
    # housing_units_per_year(), unable to resolve Server Issue
    average_gross_rent(),
    average_sales_price()
)

most_expensive = pn.Column(
    "# Statistics of Top 10 Most Expensive Neighborhoods",
    top_most_expensive_neighborhoods(),
    most_expensive_neighborhoods_rent_sales()
)

# Create the dashboard
sanfran_rental_price_dashboard = pn.Tabs(
    ('Geospatial', geospatial),
    ('Parallels', parallels),
    ('By Neighborhood', by_neighborhood),
    ('Averages', averages),
    ("Most Expensive", most_expensive)
)

## Serve the Panel Dashboard

In [132]:
# Serve the dashboard
sanfran_rental_price_dashboard.show()

Launching server at http://localhost:62750


# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [120]:
# housing_units_per_year()

In [121]:
# average_gross_rent()

In [122]:
# average_sales_price()

In [123]:
# average_price_by_neighborhood()

In [124]:
# top_most_expensive_neighborhoods()

In [125]:
# most_expensive_neighborhoods_rent_sales()

In [126]:
# neighborhood_map().show()

In [127]:
# parallel_categories()

In [128]:
# parallel_coordinates()

In [129]:
# sunburst()